## Prepare

In [126]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py

--2018-07-30 15:58:20--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 54.182.2.254, 54.182.2.101, 54.182.2.68, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|54.182.2.254|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9400 (9.2K) [application/octet-stream]
Saving to: <<bm_preproc.py.1>>

bm_preproc.py.1     100%[===================>]   9.18K  --.-KB/s    in 0s      

2018-07-30 15:58:20 (89.6 MB/s) - <<bm_preproc.py.1>> saved [9400/9400]



In [127]:
from bm_preproc import BoyerMoore

In [128]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [129]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2018-07-30 15:58:26--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 54.182.2.254, 54.182.2.101, 54.182.2.68, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|54.182.2.254|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: <<chr1.GRCh38.excerpt.fasta.1>>

chr1.GRCh38.excerpt 100%[===================>] 791.12K   124KB/s    in 6.7s    

2018-07-30 15:58:33 (118 KB/s) - <<chr1.GRCh38.excerpt.fasta.1>> saved [810105/810105]



In [130]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):
        match = True
        for j in range(len(p)):
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences

In [131]:
def naive_with_count(p, t):
    occurrences = []
    alignments = 0
    comparisons = 0
    for i in range(len(t) - len(p) + 1):
        match = True
        alignments += 1
        for j in range(len(p)):
            comparisons += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences, alignments, comparisons

def boyer_moore_with_count(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    alignments = 0
    comparisons = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        alignments += 1
        for j in range(len(p)-1, -1, -1):
            comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, alignments, comparisons

In [132]:
chr1 = readGenome('chr1.GRCh38.excerpt.fasta')

## Questions

In [133]:
print naive_with_count('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG', chr1)[1]

799954


In [22]:
print naive_with_count('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG', chr1)[2]

984143


In [134]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
p_bm = BoyerMoore(p)
print boyer_moore_with_count(p, p_bm, chr1)[1]

127974


In [1]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

--2018-07-30 13:20:51--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 54.182.2.254, 54.182.2.12, 54.182.2.101, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|54.182.2.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971 [application/octet-stream]
Saving to: <<kmer_index.py>>

kmer_index.py       100%[===================>]     971  --.-KB/s    in 0s      

2018-07-30 13:20:53 (34.3 MB/s) - <<kmer_index.py>> saved [971/971]



In [41]:
from kmer_index import Index
index = Index(chr1, 8)

In [122]:
import re
from math import ceil

def approximate_match(index, p, t, n):
    """
    index - 8-mer index
    p     - pattern
    t     - text
    n     - number of possible mismatches 
    """
    if index.k != 8:
        raise ValueError("index must be 8-mer")
    if (len(p) % 8) != 0:
        raise ValueError("the lengths of p should be factor of 8")

    segment_length = int(len(p) / (n + 1.))
    all_matches = set()
    hits = set()
    for i in range(n + 1):
        start = i * segment_length
        end = start + segment_length
        matches = index.query(p[start:end])
        hits.update(matches)
        for m in matches:
            if (m < start) or (m - start + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if p[j] != t[m - start + j]:
                    mismatches += 1
                    if mismatches > n:
                        break
            for j in range(end, len(p)):
                if p[j] != t[m - start + j]:
                    mismatches += 1
                    if mismatches > n:
                        break   
            if mismatches <= n:
                print m - start
                print mismatches
                print p
                print t[m - start:m - start + len(p)]
                print
                all_matches.add(m - start)  
    print len(hits)
    return all_matches

# 90
print len(approximate_match(index, 'GGCGCGGTGGCTCACGCCTGTAAT', chr1, 2))


56922
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

84641
1
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCATGCCTGTAAT

147558
1
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCATGCCTGTAAT

160729
1
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACACCTGTAAT

191452
1
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGTTCACGCCTGTAAT

262042
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

364263
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

657496
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

681737
1
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCATGCCTGTAAT

717706
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

56922
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

160162
2
GGCGCGGTGGCTCACGCCTGTAAT
GGCACGGTGGCTCACGCATGTAAT

262042
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

273669
1
GGCGCGGTGGCTCACGCCTGTAAT
GGTGCGGTGGCTCACGCCTGTAAT

364263
0
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCGGTGGCTCACGCCTGTAAT

421221
2
GGCGCGGTGGCTCACGCCTGTAAT
GGCGCAATGGCTCACGCCTGTAAT

429299
1
GGCGCGGTGGCTCACGCCTGTAAT
AGCGCGGTG

In [109]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, subseq):
        """ Return index hits for first subseq of p """
#         subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [110]:
sindex = SubseqIndex(chr1, 8, 3)

In [113]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
span = 22
hits = set()
for i in range(len(p) - span + 1):
    subseq = p[i:i+span:3]
    matches = sindex.query(subseq)
    hits.update(matches)

print len(hits)


79
